In [ ]:
import json
import toml

# Sample JSON string
json_str = '''
{
  "title": "Example Config",
  "owner": {
    "name": "Alice",
    "dob": "1990-01-01T00:00:00Z",
    "age": 34,
    "active": true
  },
  "servers": [
    {"id": 1, "ip": "192.168.1.1", "region": "北京"},
    {"id": 2, "ip": "10.0.0.1", "region": "上海"}
  ]
}
'''

test_str = '''
{
  "best_flights": [
    {
      "flights": [
        {
          "departure_airport": {
            "name": "String - Departure airport name",
            "id": "String - Departure airport code",
            "time": "String - Departure time"
          },
          "arrival_airport": {
            "name": "String - Arrival airport name",
            "id": "String - Arrival airport code",
            "time": "String - Arrival time"
          },
          "duration": "Integer - Flight duration, in minutes",
          "airplane": "String - Airplane model",
          "airline": "String - Name of the airline",
          "airline_logo": "String - URL to the logo of the airline",
          "travel_class": "String - Travel class",
          "flight_number": "String - The flight number",
          "extensions": "Array - A list of flight features",
          "ticket_also_sold_by": "Array - A list of other sellers",
          "legroom": "String - The legroom, including unit",
          "overnight": "Boolean - Returns 'true' if the flight is overnight",
          "often_delayed_by_over_30_min": "Boolean - Returns 'true' if the flight is often delayed by 30+ min.",
          "plane_and_crew_by": "String - Information about operating airline that provides the plane and the crew"
        }
      ],
      "layovers": [
        {
          "duration": "Integer - Duration of the layover, in minutes",
          "name": "String - Name of the airport for the layover",
          "id": "String - The airport code for the layover",
          "overnight": "Boolean - Returns 'true' if the layover is overnight"
        }
      ],
      "total_duration": "Integer - Total minutes of all flights and layovers",
      "carbon_emissions": {
        "this_flight": "Integer - The amount of carbon emissions of the flight, in grams",
        "typical_for_this_route": "Integer - Typical amount of carbon emissions for the route, in grams",
        "difference_percent": "Integer - The carbon emissions difference between the flight and typical value, in percent"
      },
      "price": "Integer - This ticket price in the selected currency, the default currency is USD",
      "type": "String - The type of the flight. It's a reflection of the request parameter `type`",
      "airline_logo": "String - URL to the logo of mixed airlines",
      "extensions": "Array - A list of features of the entire flight",
      "departure_token": "String - The token for retrieving returning flights when the flight type is 'Round trip'",
      "booking_token": "String - The token for retrieving booking options for selected flights"
    }
  ],
  "price_insights": {
    "lowest_price": "Integer - The lowest price among the returned flights",
    "price_level": "String - Price level of the lowest_price",
    "typical_price_range": "Array - A two-integer array, indicating the low bound and high bound of 'typical' price of this flight",
    "price_history": "Array - A two-integer array, where the first element is the timestamp, and the second one is the price"
  },
  "airports": [
    {
      "departure": [
        {
          "airport": {
            "name": "String - Departure airport name",
            "id": "String - Departure airport code"
          },
          "city": "String - Departure city",
          "country": "String - Departure country name",
          "country_code": "String - Departure country code",
          "image": "String - URL to the image of the departure city",
          "thumbnail": "String - URL to the thumbnail image of the departure city"
        }
      ],
      "arrival": [
        {
          "airport": {
            "name": "String - Arrival airport name",
            "id": "String - Arrival airport code"
          },
          "city": "String - Arrival city",
          "country": "String - Arrival country name",
          "country_code": "String - Arrival country code",
          "image": "String - URL to the image of the arrival city",
          "thumbnail": "String - URL to the thumbnail image of the arrival city"
        }
      ]
    }
  ]
}
'''

# Step 1: Parse JSON string to Python dict
data = json.loads(test_str)

# Step 2: Convert Python dict to TOML string
toml_str = toml.dumps(data)

print(toml_str)

In [ ]:
from serpapi import GoogleSearch

params = {
  "engine": "google_flights",
  "departure_id": "PEK",
  "arrival_id": "AUS",
  "outbound_date": "2025-05-25",
  "return_date": "2025-05-29",
  "currency": "USD",
  "hl": "en",
  "api_key": "8f3275b443423d97c1bcc5fd8453f3e3b260bf73cdd2c3282c74da4d8c4e06a2"
}

search = GoogleSearch(params)
results = search.get_dict()

In [11]:
from server.google_flights import FlightsResponseModel

import json 

with open('server/sample_response.json', 'r') as f:
    obj = json.load(f)
    obj['test'] = 1
    a = FlightsResponseModel(**obj)

In [ ]:
a.test

In [ ]:
a.best_flights[0].model_dump()


In [7]:
output_data = {}
if a.best_flights:
    output_data["best_flights"] = [
        flight.model_dump(exclude_none=True) for flight in a.best_flights
    ]
if a.other_flights:
    output_data["other_flights"] = [
        flight.model_dump(exclude_none=True) for flight in a.other_flights
    ]

In [ ]:
import toml 
print(toml.dumps(output_data))

In [4]:
from mcp.client.streamable_http import streamablehttp_client
from mcp import ClientSession

async def main():
    async with streamablehttp_client("http://localhost:3002/mcp") as (read_stream, write_stream, _):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tools = await session.list_tools()
            print(tools)
            res = await session.call_tool("execute_python_code", {"code": "print('Hello, world!')"})
            print(res)

await main()



meta=None nextCursor=None tools=[Tool(name='execute_python_code', description='Executes the given Python code string and returns its output. the executed code should print its input and output if has any\n\nArgs:\n    code: The Python code to execute.\n', inputSchema={'properties': {'code': {'title': 'Code', 'type': 'string'}}, 'required': ['code'], 'title': 'execute_python_codeArguments', 'type': 'object'}, annotations=None), Tool(name='execute_bash_script', description='Executes the given bash script string and returns its output.\nAttempts to filter out some potentially harmful commands.\nthe executed script should print its input and output if has any\n\nArgs:\n    script: The bash script to execute.\n', inputSchema={'properties': {'script': {'title': 'Script', 'type': 'string'}}, 'required': ['script'], 'title': 'execute_bash_scriptArguments', 'type': 'object'}, annotations=None)]
meta=None content=[TextContent(type='text', text='Execution successful:\nOutput:\nHello, world!\n', a